In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transform
from torchvision.transforms import Compose

In [2]:
_ = torch.manual_seed(42)

In [3]:
# hyperparameters
batch_size = 64
sequence_length = 28
input_size = 28
hidden_size = 64
num_layers = 1
num_classes = 10
num_epochs = 3
learning_rate = 0.001

## Loading the MNIST Dataset


In [4]:
mnist_train  = torchvision.datasets.MNIST(
    root='./MNIST/',
    train=True,
    transform=transform.ToTensor(),
    download=True
)

mnist_test  = torchvision.datasets.MNIST(
    root='./MNIST/',
    train=False,
    transform=transform.ToTensor(),
    download=True
)

dataloader_train = DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True)

dataloader_test = DataLoader(
    dataset=mnist_test,
    batch_size=batch_size,
    shuffle=True)

## Creating the RNN in PyTorch

In [5]:
class RNN(nn.Module):
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            num_layers: int,
            num_classes: int,
            sequence_length: int):
        super().__init__()

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True)

        self.fc = nn.Linear(
            in_features=hidden_size * sequence_length,
            out_features=num_classes,
            bias=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x, _ = self.rnn(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x

In [6]:
model = RNN(input_size, hidden_size, num_layers, num_classes, sequence_length)
x = torch.rand(64, 1, 28, 28)
print(x.shape)

torch.Size([64, 1, 28, 28])


In [7]:
x = x.view(x.size(0), 28, 28)
print(x.shape)

y = model(x)
print(y.shape)

torch.Size([64, 28, 28])
torch.Size([64, 10])


In [8]:
model = RNN(input_size, hidden_size, num_layers, num_classes, sequence_length)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()


for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    for X, y in dataloader_train:
        optimizer.zero_grad() 
        # X = X.view(X.size(0), 28, 28) 
        X = X.squeeze(1)
        y_pred = model(X)

        loss = loss_fn(y_pred, y)

        total_loss += loss.sum().item()
        correct += (torch.argmax(y_pred, dim=1) == y).sum()
        
        loss.backward()

        optimizer.step()

    print(f'epoch: {epoch}\tloss: {total_loss/len(dataloader_train)}\tacc: {correct/len(dataloader_train)}')



epoch: 0	loss: 0.3229574835710307	acc: 57.9083137512207
epoch: 1	loss: 0.15409702121683244	acc: 60.98507308959961
epoch: 2	loss: 0.10676010557190219	acc: 61.84541702270508


In [9]:
correct = 0
with torch.no_grad():
    for X, y in dataloader_test:
        X = X.squeeze(1) 
        y_pred = model(X)
        correct += (torch.argmax(y_pred, dim=1) == y).sum()
acc = correct / len(dataloader_test)
print(acc.item())

62.0
